In [1]:
library(dplyr)
library(DBI)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
issue_con <- function() {
    # assume you are using config package and Postgres
    crd <- config::get()
    DBI::dbConnect(
        drv = RPostgres::Postgres(),
        host = crd$host,
        port = crd$port,
        dbname = crd$dbname,
        user = crd$user,
        password = crd$password
    )
}

release_cons <- function(search_from = ls(".GlobalEnv"), except = NULL) {
    purrr::map_chr(
        .x = search_from %>% base::setdiff(except),
        .f = function(x) {
            ret <- ""
            if (x %>% get() %>%
                {
                    inherits(., "PqConnection") & length(.) == 1
                }) {
                ret <- tryCatch(
                    {
                        x %>%
                            get() %>%
                            DBI::dbDisconnect()
                        return(x)
                    },
                    error = function(e) {
                        msg <- paste("Invalid PqConnection class object is provided:", x)
                        message(msg)
                    },
                    warning = function(e) {
                        return("")
                    }
                )
            }
            return(ret)
        }
    ) %>% .[. != ""]
}

In [3]:
refresh_con <- function(){
	release_cons()
	issue_con()
}

In [4]:
dbCreateView <- function(con, name, sql, or_replace = FALSE, temporary = FALSE) {
    if (sql %>% inherits(c("character", "SQL")) %>% isFALSE()) stop("provided sql string is not of class character or SQL.")
    con %>%
        DBI::sqlInterpolate(
            sql = "CREATE ?or ?tmp VIEW ?name AS ?q",
            or = dplyr::if_else(or_replace, "OR REPLACE", "") %>% DBI::SQL(),
            tmp = dplyr::if_else(temporary, "TEMPORARY", "") %>% DBI::SQL(),
            name = DBI::dbQuoteIdentifier(., name),
            q = sql %>% DBI::SQL()
        ) %>%
        DBI::dbExecute(con, .)
    invisible(TRUE) %>% return()
}

dbDropView <- function(con, name, if_exists = FALSE, cascade = FALSE) {
    before <- con %>%
        dbListViews() %>%
        length()
    con %>%
        DBI::sqlInterpolate(
            sql = "DROP VIEW ?if_ex ?name ?cas_res",
            if_ex = dplyr::if_else(if_exists, "IF EXISTS", "") %>% DBI::SQL(),
            name = DBI::dbQuoteIdentifier(con, name),
            cas_res = dplyr::if_else(cascade, "CASCADE", "RESTRICT") %>% SQL()
        ) %>%
        DBI::dbExecute(con, .)
    before - (con %>% dbListViews() %>% length()) %>% invisible()
}

In [6]:
dbListViews <- function(con, permanent = TRUE, temporary = TRUE) {
    ret <- character()
    # this returns all views, temporary or not
    list_views <- function(x) {
        x %>%
            dbListTables() %>%
            setdiff(
                x %>%
                    DBI::dbGetQuery(
                        statement = "SELECT tablename FROM pg_tables WHERE schemaname = current_schema()"
                    ) %>% pull(1)
            )
    }
    # retruns only non-temporary views
    parm_views <- function(x) {
        x %>%
            dbGetQuery(
                statement = "SELECT viewname FROM pg_views WHERE schemaname = current_schema()"
            ) %>%
            pull(1)
    }
    # we can't use dplyr::case_when here, which requires the returned values be the same type and the same length.
    if (permanent & temporary) {
        ret <- con %>% list_views()
    } else if (permanent) {
        ret <- con %>% parm_views()
    } else if (temporary) {
        ret <- con %>%
            list_views() %>%
            setdiff(con %>% parm_views())
    }
    ret %>% return()
}

In [10]:
con <- refresh_con()
mtcars_inline <- con %>% dbplyr::copy_inline(mtcars)
mtcars_inline_sql <- mtcars_inline %>% dbplyr::sql_render()
con %>% dbCreateView(
	name = 'mtcars_view',
	sql = mtcars_inline_sql,
	or_replace = TRUE
)

In [16]:
# view on copy_inline() survives after disconnection
con <- refresh_con()
con %>% dbListTables()
con %>% tbl('mtcars_view') %>% count()
con %>% dbDropView('mtcars_view')

[1] "customer"    "category"    "geocode"     "product"     "receipt"    
[6] "store"       "mtcars_view"

# Source:   SQL [1 x 1]
# Database: postgres  [guest@localhost:5432/knock100]
        n
  <int64>
1      32

[1] 1

In [20]:
# case: copy_to()
con %>% copy_to(
	df = mtcars,
	name = 'mtcars_table'
)
con %>% dbListTables()
con <- refresh_con()
con %>% dbListTables()

[1] "customer"     "category"     "geocode"      "product"      "receipt"     
[6] "store"        "mtcars_table"

[1] "customer" "category" "geocode"  "product"  "receipt"  "store"

In [23]:
# 
con <- refresh_con()
con %>% copy_to(
	df = mtcars,
	name = 'mtcars_table'
)
con %>% dbCreateView(
	name = 'mtcars_view',
	sql = "SELECT * FROM mtcars_table"
)
con %>% dbListTables()
con <- refresh_con()
con %>% dbListTables()

NOTICE:  view "mtcars_view" will be a temporary view




[1] "customer"     "category"     "mtcars_table" "mtcars_view"  "geocode"     
[6] "product"      "receipt"      "store"

[1] "customer" "category" "geocode"  "product"  "receipt"  "store"

In [27]:
con %>%
    tbl("receipt") %>%
    filter(amount >= 1000) %>%
    copy_to(
		dest = con,
	 df = .,
	 name = 'receipt_copied'
	 ) # from tbl_lazy to table
con %>% dbListTables()
con %>% dbDisconnect()
con <- issue_con()
con %>% dbListTables()

[1] "customer"       "receipt_copied" "category"       "geocode"       
[5] "product"        "receipt"        "store"

[1] "customer" "category" "geocode"  "product"  "receipt"  "store"